# Машинное обучение без учителя.
автор Д.В.Яценко

## Кластеризация

In [42]:
import time
import warnings
import numpy as np
import matplotlib.pyplot as plt
from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

### Создадим наборы данных

In [43]:
np.random.seed(0)

# ============
# Создание набора данных для работы (набора точек). Размер выбран так, чтобы
# можно было заметить различия в работе методов, связанные с количество признаков,
# но не настолько большим, чтобы исполнение кода длилось слишком долго.
# ============
n_samples = 1500
noisy_circles = datasets.make_circles(n_samples=n_samples, factor=.5,
                                      noise=.05)
noisy_moons = datasets.make_moons(n_samples=n_samples, noise=.05)
blobs = datasets.make_blobs(n_samples=n_samples, random_state=8)
no_structure = np.random.rand(n_samples, 2), None

# Создание распределлённых данных
random_state = 170
X, y = datasets.make_blobs(n_samples=n_samples, random_state=random_state)
transformation = [[0.6, -0.6], [-0.4, 0.8]]
X_aniso = np.dot(X, transformation)
aniso = (X_aniso, y)

# blobs with varied variances
varied = datasets.make_blobs(n_samples=n_samples,
                             cluster_std=[1.0, 2.5, 0.5],
                             random_state=random_state)
# ============
# Установка параметров кластеров
# ============
plt.rcParams["figure.figsize"] = (30,20)
plt.figure()

plt.subplots_adjust(left=.02, right=.98, bottom=.001, top=.95, wspace=.05,
                    hspace=.01)

plot_num = 1

default_base = {'quantile': .3,
                'eps': .3,
                'damping': .9,
                'preference': -200,
                'n_neighbors': 10,
                'n_clusters': 3,
                'min_samples': 20,
                'xi': 0.05,
                'min_cluster_size': 0.1}

datasets = [
    (noisy_circles, {'damping': .77, 'preference': -240,
                     'quantile': .2, 'n_clusters': 2,
                     'min_samples': 20, 'xi': 0.25}),
    (noisy_moons, {'damping': .75, 'preference': -220, 'n_clusters': 2}),
    (varied, {'eps': .18, 'n_neighbors': 2,
              'min_samples': 5, 'xi': 0.035, 'min_cluster_size': .2}),
    (aniso, {'eps': .15, 'n_neighbors': 2,
             'min_samples': 20, 'xi': 0.1, 'min_cluster_size': .2}),
    (blobs, {}),
    (no_structure, {})]

<Figure size 3000x2000 with 0 Axes>

In [45]:
def print_metrics(model_name,X,y):
    silhouette = silhouette_score(X, y)
    calinski_harabasz = calinski_harabasz_score(X,y)
    davies_bouldin = davies_bouldin_score(X, y)
    print(f"Метрики качества кластеризации для модели {model_name}\n\t silhouette:{silhouette}\n\t calinski_harabasz: {calinski_harabasz}\n\t davies_bouldin:{davies_bouldin}\n======")

In [ ]:
for i_dataset, (dataset, algo_params) in enumerate(datasets):
    # изменение параметров на специфичные для набора данных значения
    params = default_base.copy()
    params.update(algo_params)

    X, y = dataset

    # нормализация набора данных 
    X = StandardScaler().fit_transform(X)

    # оценка ширины полосы для метода "mean shift"
    bandwidth = cluster.estimate_bandwidth(X, quantile=params['quantile'])

    # расчёт матрицы связности для метода AgglomerativeClustering
    connectivity = kneighbors_graph(
        X, n_neighbors=params['n_neighbors'], include_self=False)
    # симметричность
    connectivity = 0.5 * (connectivity + connectivity.T)

    # ============
    # Создание кластеров
    # ============
    ms = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
    two_means = cluster.MiniBatchKMeans(n_clusters=params['n_clusters'])
    ward = cluster.AgglomerativeClustering(
        n_clusters=params['n_clusters'], linkage='ward',
        connectivity=connectivity)
    spectral = cluster.SpectralClustering(
        n_clusters=params['n_clusters'], eigen_solver='arpack',
        affinity="nearest_neighbors")
    dbscan = cluster.DBSCAN(eps=params['eps'])
    optics = cluster.OPTICS(min_samples=params['min_samples'],
                            xi=params['xi'],
                            min_cluster_size=params['min_cluster_size'])
    affinity_propagation = cluster.AffinityPropagation(
        damping=params['damping'], preference=params['preference'])
    average_linkage = cluster.AgglomerativeClustering(
        linkage="average", # affinity="cityblock",
        n_clusters=params['n_clusters'], connectivity=connectivity)
    birch = cluster.Birch(n_clusters=params['n_clusters'])
    gmm = mixture.GaussianMixture(
        n_components=params['n_clusters'], covariance_type='full')

    clustering_algorithms = (
        ('MiniBatch\nKMeans', two_means),
        ('Affinity\nPropagation', affinity_propagation),
        ('MeanShift', ms),
        ('Spectral\nClustering', spectral),
        ('Ward', ward),
        ('Agglomerative\nClustering', average_linkage),
        ('DBSCAN', dbscan),
        ('OPTICS', optics),
        ('BIRCH', birch),
        ('Gaussian\nMixture', gmm)
    )

    for name, algorithm in clustering_algorithms:
        t0 = time.time()

        # работа с предупреждениям (ворнингами, warnings)
        with warnings.catch_warnings():
            warnings.filterwarnings(
                "ignore",
                message="the number of connected components of the " +
                "connectivity matrix is [0-9]{1,2}" +
                " > 1. Completing it to avoid stopping the tree early.",
                category=UserWarning)
            warnings.filterwarnings(
                "ignore",
                message="Graph is not fully connected, spectral embedding" +
                " may not work as expected.",
                category=UserWarning)
            algorithm.fit(X)

        t1 = time.time()
        if hasattr(algorithm, 'labels_'):
            y_pred = algorithm.labels_.astype(int)
        else:
            y_pred = algorithm.predict(X)
        if i_dataset ==2:   
            print_metrics(name,X,y_pred)
        plt.subplot(len(datasets), len(clustering_algorithms), plot_num)
        if i_dataset == 0:
            plt.title(name, size=18)

        colors = np.array(list(islice(cycle(['#377eb8', '#ff7f00', '#4daf4a',
                                             '#f781bf', '#a65628', '#984ea3',
                                             '#999999', '#e41a1c', '#dede00']),
                                      int(max(y_pred) + 1))))
        # добавление чёрного цвета к объектам, которые были определены, как не относящиеся ни к одному кластеру
        colors = np.append(colors, ["#000000"])
        plt.scatter(X[:, 0], X[:, 1], s=10, color=colors[y_pred])

        plt.xlim(-2.5, 2.5)
        plt.ylim(-2.5, 2.5)
        plt.xticks(())
        plt.yticks(())
        plt.text(.99, .01, ('%.2fs' % (t1 - t0)).lstrip('0'),
                 transform=plt.gca().transAxes, size=15,
                 horizontalalignment='right')
        plot_num += 1

plt.show()

## Снижение размерности

In [24]:
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE

### Данные

In [ ]:
dataset = datasets.load_wine()
X = dataset.data
y = dataset.target

### Cтандартизация
Centering and scaling

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### Метод понижения размерности PCA

In [ ]:
pca = PCA(n_components=2)
principal_components = pca.fit_transform(X) # Позже заменить на X_scaled

In [ ]:
plt.scatter(principal_components[:,0],principal_components[:,1],c=y)

In [ ]:
clf = LogisticRegression(random_state=0).fit(principal_components, y)
clf.score(principal_components, y)

### Метод понижения размерности TSNE

In [ ]:
tsne = TSNE(n_components=2, random_state=42)
X_tsne = tsne.fit_transform(X) # Позже заменить на X_scaled

In [ ]:
plt.scatter(X_tsne[:,0],X_tsne[:,1],c=y)

In [ ]:
clf = LogisticRegression(random_state=0).fit(X_tsne, y)
clf.score(X_tsne, y)